# Feature Engineering

El objetivo de este notebook es enriquecer el dataset user_reviews con una nueva columna llamada 'sentiment_analysis'. Esta columna contendrá el resultado de aplicar un análisis de sentimiento con NLP a las reseñas de los juegos escritas por los usuarios. De esta manera, podremos explorar la opinión de los usuarios sobre los diferentes juegos. Además, vamos a hacer un merge entre los distintos datasets para obtener una base de datos única y más eficiente para cada consulta, que nos permitirá optimizar el rendimiento de la API y reducir el tamaño de nuestra base de datos.

## 0 Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y/o modulos necesarios para nuestro proceso de feature engineering y establecemos configuraciones globales de ser requerido.

In [71]:
import sys
import pandas as pd
import textblob
from textblob import TextBlob

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"TextBlob version: {textblob.__version__}")

System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Pandas version: 1.5.3
TextBlob version: 0.17.1


## 1 Extracción

En esta sección, extraemos los datos de los archivos `steam_games`, `user_items` y `user_reviews` que estan en formato parquet.

### 1.1 Extracción de los datos

Creamos una función que lee cada archivo desde su directorio y lo carga a un DataFrame de `pandas`.

In [2]:
# Cargamos los archivos parquet
def read_parquet_files(parquet_files):
    dataframes = {}
    for name in parquet_files:
        dataframes[name] = pd.read_parquet(f'data/interim/{name}.parquet', engine='pyarrow')
    return dataframes

parquet_files = ['steam_games','user_items', 'user_reviews']
dataframes = read_parquet_files(parquet_files)

# Convertimos a df.
df_steam_games = dataframes['steam_games']
df_user_items = dataframes['user_items']
df_user_reviews = dataframes['user_reviews']

## 2 Análisis de sentimiento

El análisis de sentimiento consiste en asignar una etiqueta numérica a cada reseña, según el tono o la actitud que expresa el texto. Usaremos la siguiente escala:

* 0: si la reseña es negativa, es decir, si el usuario muestra insatisfacción, disgusto o decepción con el juego.
* 1: si la reseña es neutral, es decir, si el usuario muestra indiferencia, objetividad o ausencia de emoción con el juego.
* 2: si la reseña es positiva, es decir, si el usuario muestra satisfacción, gusto o admiración con el juego.



### 2.1 Función `sentiment_analysis`

Para realizar el análisis de sentimiento con NLP a las reseñas de los juegos, crearemos una función usando la librería TextBlob que se considera facil de usar y muy intuitiva. Usaremos la polaridad que es una medida numérica que indica si el texto es negativo o positivo, según el tono o la actitud que expresa. La polaridad varía entre -1 y 1, donde -1 significa muy negativo, 0 significa neutro y 1 significa muy positivo.

In [3]:
def sentiment_analysis(review):
    # Si la reseña está ausente, retorna 1 (neutral)
    if pd.isnull(review):
        return 1

    # Calcula la polaridad de la reseña usando TextBlob
    polarity = TextBlob(review).sentiment.polarity

    # Retorna 0 (malo) si la polaridad es menor que 0, 2 (positivo) si la polaridad es mayor que 0, y 1 (neutral) en caso contrario
    if polarity < 0:
        return 0
    elif polarity > 0:
        return 2
    else:
        return 1

* Aplicamos la función a la columna 'review'.

In [4]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(sentiment_analysis)

* Veamos algunos ejemplos:

In [5]:
df_user_reviews[['review','sentiment_analysis']].sample(5)

,review,sentiment_analysis
39236,"Great medieval fighting techniques, i espicail...",2
34342,Coming back to this game after eight years was...,2
55586,"Very fun to play, but what ever you do, DO NOT...",0
34297,If you liked the first game you will like this...,2
43465,(Español-Spanish) Buen juego para alguien que ...,1


### 2.2 Eliminación de la columna `review`

La nueva columna ‘sentiment_analysis’ reemplazará a la columna ‘review’ en el dataset 'user_reviews', para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [6]:
df_user_reviews.drop('review', axis=1, inplace=True)
df_user_reviews.head()

,item_id,recommend,user_id,posted_year,sentiment_analysis
0,1250,True,76561197970982479,2011,2
1,22200,True,76561197970982479,2011,2
2,43110,True,76561197970982479,2011,2
3,251610,True,js41637,2014,2
4,227300,True,js41637,2013,0


In [7]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58431 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             58431 non-null  object
 1   recommend           58431 non-null  bool  
 2   user_id             58431 non-null  object
 3   posted_year         58431 non-null  object
 4   sentiment_analysis  58431 non-null  int64 
dtypes: bool(1), int64(1), object(3)
memory usage: 2.3+ MB


## 3 Diseño y estructura de las bases de datos para los endpoints de la API

En esta sección, nuestro objetivo es crear diferentes dataset para las funciones que se usarán en los endpoints de la API. De esta manera, podremos acceder a los datos que necesitamos de forma rápida y eficiente, sin tener que cargar toda la información para así, optimizar el rendimiento de la API.

### 3.1 Endpoint 1

def **developer( *`desarrollador` : str* )**:
    `Cantidad` de items y `porcentaje` de contenido Free por año según empresa desarrolladora.
Ejemplo de retorno:

| Año  | Cantidad de Items | Contenido Free  |
|------|-------------------|------------------|
| 2023 | 50                | 27%              |
| 2022 | 45                | 25%              |
| xxxx | xx                | xx%              |


* Para esta función solo necesitamos las columnas:
'developer', 'price', 'item_id' y 'release_year' del dataset 'steam_games'

In [8]:
df_endpoint1 = df_steam_games[['developer', 'item_id', 'price', 'release_year']]

* Selecionamos solo los juegos que tienen registrado un desarrollador y su año de lanzamiento

In [9]:
df_endpoint1 = df_endpoint1[(df_endpoint1['developer'] != 'unknown') & (df_endpoint1['release_year'] != 'unknown')]
df_endpoint1 = df_endpoint1.reset_index(drop=True)
df_endpoint1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28714 entries, 0 to 28713
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   developer     28714 non-null  object 
 1   item_id       28714 non-null  object 
 2   price         28714 non-null  float64
 3   release_year  28714 non-null  object 
dtypes: float64(1), object(3)
memory usage: 897.4+ KB


### 3.2 Endpoint 2

+ def **userdata( *`User_id` : str* )**:
    Debe devolver `cantidad` de dinero gastado por el usuario, el `porcentaje` de recomendación en base a `reviews.recommend == true` y `cantidad de items`.

Ejemplo de retorno:
```json
{
  "Usuario X" : "us213ndjss09sdf",
  "Dinero gastado": 200,
  "% de recomendación": 0.2,
  "cantidad de items": 5
}
```


Para esta función solo necesitamos las siguientes columnas:

In [10]:
steam_games_columns = ['item_id','price']
user_items_columns = ['item_id','user_id','items_count']
user_reviews_columns = ['item_id','user_id', 'recommend']

* Primero, creamos subsets de los dataframes con solo las columnas necesarias

In [11]:
df_games_subset = df_steam_games[steam_games_columns]
df_reviews_subset = df_user_reviews[user_reviews_columns]
df_items_subset = df_user_items[user_items_columns]

* Luego, hacemos un merge entre 'steam_games' y 'user_items' en la columna 'item_id'

In [12]:
df_games_price = pd.merge(df_games_subset, df_items_subset, on='item_id', how='right')

* Verificamos qué hay juegos que poseen los usuarios, pero que no tienen un registro en 'steam_games' y, por lo tanto, no tienen un precio asignado.

In [13]:
df_games_price[df_games_price['price'].isnull()]

,item_id,price,user_id,items_count
13,34440,NaN,76561197970982479,277
26,9340,NaN,76561197970982479,277
27,228200,NaN,76561197970982479,277
37,17340,NaN,76561197970982479,277
51,23120,NaN,76561197970982479,277
...,...,...,...,...
5094076,493440,NaN,76561198326700687,177
5094081,496920,NaN,76561198326700687,177
5094089,508380,NaN,76561198326700687,177
5094091,510660,NaN,76561198326700687,177


* Calculamos la media, la moda y la mediana de los precios para así, poder determinar cuál de estas medidas es la más adecuada para imputar los valores faltantes.

In [14]:
media = df_games_price['price'].mean()
moda = df_games_price['price'].mode()[0]
mediana = df_games_price['price'].median()

print(f"La media de los precios es {media:.2f}")
print(f"La moda de los precios es {moda:.2f}")
print(f"La mediana de los precios es {mediana:.2f}")
print(df_games_price['price'].describe())

La media de los precios es 12.43
La moda de los precios es 0.00
La mediana de los precios es 9.99
count    4.244831e+06
mean     1.242628e+01
std      1.338972e+01
min      0.000000e+00
25%      4.990000e+00
50%      9.990000e+00
75%      1.999000e+01
max      7.717100e+02
Name: price, dtype: float64


En base a la observación de los datos, hemos decidido imputar los valores nulos en la columna de precios con la mediana por las siguientes razones:

1. **Resistencia a valores extremos (outliers)**: La mediana es una medida de tendencia central que no se ve afectada por valores extremos. En nuestro caso, el precio de los juegos puede variar ampliamente, y algunos juegos pueden tener precios muy altos que podrían distorsionar la media. Al usar la mediana, nos aseguramos de que nuestra imputación no esté sesgada por estos valores extremos.

2. **Distribución de los datos**: La mediana es el punto medio de la distribución de los precios, lo que significa que la mitad de los juegos cuestan menos que la mediana y la otra mitad cuesta más. Al imputar con la mediana, estamos eligiendo un valor que es representativo de la distribución general de los precios.

3. **Interpretación intuitiva**: La mediana tiene una interpretación intuitiva como el "precio típico" de un juego, lo cual puede ser útil para nuestro análisis.

Por lo tanto, aunque la elección del método de imputación puede depender del contexto y del conocimiento del dominio, en este caso, la mediana parece ser una opción razonable dada la naturaleza de nuestros datos.

* Imputamos los valores faltantes con la mediana.

In [15]:
df_games_price['price'].fillna(df_games_price['price'].median(), inplace=True)
df_games_price['price'].isnull().sum()

0

* Agrupamos por usuario para sumar la cantidad de dinero gastado por juego para cada usuario.

In [16]:
df_user_spend = df_games_price.groupby('user_id')['price'].sum().reset_index()
df_user_spend.columns = ['user_id', 'total_spend']
df_user_spend.head()

,user_id,total_spend
0,--000--,502.67
1,--ace--,226.76
2,--ionex--,149.88
3,-2SV-vuLB-Kg,587.34
4,-404PageNotFound-,1883.94


* Hacemos un merge entre 'games_price' y el subset de 'user_reviews' en las columnas 'user_id' e 'item_id' para anexar la recomendacion de cada juego.

In [17]:
df_endpoint2 = pd.merge(df_games_price, df_reviews_subset, on=['user_id','item_id'])
df_endpoint2.head()

,item_id,price,user_id,items_count,recommend
0,22200,9.99,76561197970982479,277,True
1,1250,19.99,76561197970982479,277,True
2,43110,9.99,76561197970982479,277,True
3,227300,19.99,js41637,888,True
4,239030,9.99,js41637,888,True


* Luego, hacemos un último merge con 'user_spend' para anexar la cantidad total de dinero gastado por el usuario.

In [18]:
df_endpoint2 = pd.merge(df_endpoint2, df_user_spend, on=['user_id'])
df_endpoint2.head()

,item_id,price,user_id,items_count,recommend,total_spend
0,22200,9.99,76561197970982479,277,True,3878.85
1,1250,19.99,76561197970982479,277,True,3878.85
2,43110,9.99,76561197970982479,277,True,3878.85
3,227300,19.99,js41637,888,True,9612.00
4,239030,9.99,js41637,888,True,9612.00


* Por último, eliminamos la columna 'item_id' que ya no necesitamos.

In [19]:
df_endpoint2.drop(['item_id','price'], axis=1, inplace=True)
df_endpoint2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44187 entries, 0 to 44186
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      44187 non-null  object 
 1   items_count  44187 non-null  int64  
 2   recommend    44187 non-null  bool   
 3   total_spend  44187 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(1)
memory usage: 1.4+ MB


### 3.3 Endpoint 3

+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno:
```json
{
  "Usuario con más horas jugadas para Género X" : "us213ndjss09sdf",
  "Horas jugadas": [
    {"Año": 2013, "Horas": 203},
    {"Año": 2012, "Horas": 100},
    {"Año": 2011, "Horas": 23}
  ]
}


Para crear un solo dataset que pueda ser utilizado para esta función , necesitaríamos combinar 'df_steam_games' con df_user_items' de tal manera que tengamos toda la información necesaria en un solo lugar.

In [50]:
steam_games_columns = ['item_id','genres','release_year']
user_items_columns = ['item_id','user_id', 'playtime_forever']

* Primero, seleccionamos solo las columnas necesarias.

In [51]:
df_games_subset = df_steam_games[steam_games_columns]
df_items_subset = df_user_items[user_items_columns]

* Luego, hacemos un merge entre 'steam_games' y 'user_items' en la columna 'item_id'

In [52]:
df_endpoint3 = pd.merge(df_games_subset, df_items_subset, on='item_id')
df_endpoint3.head()

,item_id,genres,release_year,user_id,playtime_forever
0,282010,"[Racing, Action, Indie]",1997,UTNerd24,0.083333
1,282010,"[Racing, Action, Indie]",1997,I_DID_911_JUST_SAYING,0.000000
2,282010,"[Racing, Action, Indie]",1997,76561197962104795,0.000000
3,282010,"[Racing, Action, Indie]",1997,r3ap3r78,0.000000
4,282010,"[Racing, Action, Indie]",1997,saint556,0.216667


In [56]:
df_endpoint3_exploited = df_endpoint3.explode('genres')
df_endpoint3_exploited

,item_id,genres,release_year,user_id,playtime_forever
0,282010,Racing,1997,UTNerd24,0.083333
0,282010,Action,1997,UTNerd24,0.083333
0,282010,Indie,1997,UTNerd24,0.083333
1,282010,Racing,1997,I_DID_911_JUST_SAYING,0.000000
1,282010,Action,1997,I_DID_911_JUST_SAYING,0.000000
...,...,...,...,...,...
4244829,80,Simulation,2004,76561198283312749,0.150000
4244830,80,Action,2004,76561198312638244,1.650000
4244830,80,Strategy,2004,76561198312638244,1.650000
4244830,80,Adventure,2004,76561198312638244,1.650000


* Seleccionamos los 10 generos mas populares para dismunuir el tamaño del dataframe y optimizar el rendimiento en las consultas de la API.

In [57]:
top_10_popular_genres = ['Action', 'Adventure', 'Strategy', 'Indie', 'RPG', 'Simulation', 'Massively Multiplayer', 'Casual', 'Racing', 'Sports']
df_endpoint3_exploited = df_endpoint3_exploited[df_endpoint3_exploited['genres'].isin(top_10_popular_genres)]
df_endpoint3_exploited = df_endpoint3_exploited[df_endpoint3_exploited['release_year'] != 'unknown']
df_endpoint3_exploited

,item_id,genres,release_year,user_id,playtime_forever
0,282010,Racing,1997,UTNerd24,0.083333
0,282010,Action,1997,UTNerd24,0.083333
0,282010,Indie,1997,UTNerd24,0.083333
1,282010,Racing,1997,I_DID_911_JUST_SAYING,0.000000
1,282010,Action,1997,I_DID_911_JUST_SAYING,0.000000
...,...,...,...,...,...
4244829,80,Simulation,2004,76561198283312749,0.150000
4244830,80,Action,2004,76561198312638244,1.650000
4244830,80,Strategy,2004,76561198312638244,1.650000
4244830,80,Adventure,2004,76561198312638244,1.650000


* Agrupamos para obtener el tiempo total de juego por año, genero y usuario.

In [61]:
df_endpoint3 = df_endpoint3_exploited.groupby(['user_id', 'release_year', 'genres'])['playtime_forever'].sum().reset_index()
df_endpoint3

,user_id,release_year,genres,playtime_forever
0,--000--,2006,Action,15.416667
1,--000--,2006,Adventure,15.416667
2,--000--,2006,Indie,15.416667
3,--000--,2006,Massively Multiplayer,15.416667
4,--000--,2006,Simulation,15.416667
...,...,...,...,...
4118669,zzzmidmiss,2014,Strategy,0.266667
4118670,zzzmidmiss,2016,Casual,0.000000
4118671,zzzmidmiss,2016,Indie,0.000000
4118672,zzzmidmiss,2016,Simulation,0.000000


In [62]:
df_endpoint3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4118674 entries, 0 to 4118673
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   release_year      object 
 2   genres            object 
 3   playtime_forever  float64
dtypes: float64(1), object(3)
memory usage: 125.7+ MB


* Creamos un dataframe a partir de una tabla pivote que tiene como índice 'user_id'y 'release_year', como columnas 'genres' y como valores únicos 'playtime_forever'.

In [63]:
df_endpoint3 = df_endpoint3.pivot_table(index=['user_id', 'release_year'], columns='genres', values='playtime_forever', fill_value=0)
df_endpoint3

genres                        Action   Adventure     Casual      Indie  \
user_id    release_year                                                  
--000--    2006            15.416667   15.416667   0.000000  15.416667   
           2009            88.816667   88.816667   0.000000   0.000000   
           2010             0.366667    0.000000   0.000000   0.366667   
           2011           108.700000  108.700000   0.000000  30.616667   
           2012          1822.516667   37.150000  30.016667  37.700000   
...                              ...         ...        ...        ...   
zzzmidmiss 2011            38.366667   38.366667   1.250000   1.750000   
           2012            98.366667   61.650000   6.083333  51.316667   
           2013             1.633333    1.750000   0.283333   1.750000   
           2014             0.266667    0.266667   0.266667   0.000000   
           2016             0.000000    0.000000   0.000000   0.000000   

genres                   Massively Multiplayer        RPG     Racing  \
user_id    release_year                                                
--000--    2006                      15.416667   0.000000   0.000000   
           2009                       0.000000   0.000000   0.000000   
           2010                       0.000000   0.000000   0.000000   
           2011                      46.050000  62.650000  46.050000   
           2012                      10.500000  29.516667   0.000000   
...                                        ...        ...        ...   
zzzmidmiss 2011                       0.266667  37.600000   0.266667   
           2012                       8.016667  45.500000   0.000000   
           2013                       0.166667   0.166667   0.000000   
           2014                       0.266667   0.000000   0.000000   
           2016                       0.000000   0.000000   0.000000   

genres                   Simulation    Sports     Strategy  
user_id    release_year                                     
--000--    2006           15.416667  0.000000     0.000000  
           2009            0.000000  0.000000     0.000000  
           2010            0.000000  0.000000     0.000000  
           2011           11.083333  0.000000    11.083333  
           2012            0.000000  7.683333  1796.400000  
...                             ...       ...          ...  
zzzmidmiss 2011            0.000000  0.000000     1.150000  
           2012            6.450000  0.000000    15.383333  
           2013            0.000000  0.000000     1.466667  
           2014            0.266667  0.266667     0.266667  
           2016            0.000000  0.000000     0.000000  

[771829 rows x 10 columns]

In [64]:
df_endpoint3.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 771829 entries, ('--000--', '2006') to ('zzzmidmiss', '2016')
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Action                 771829 non-null  float64
 1   Adventure              771829 non-null  float64
 2   Casual                 771829 non-null  float64
 3   Indie                  771829 non-null  float64
 4   Massively Multiplayer  771829 non-null  float64
 5   RPG                    771829 non-null  float64
 6   Racing                 771829 non-null  float64
 7   Simulation             771829 non-null  float64
 8   Sports                 771829 non-null  float64
 9   Strategy               771829 non-null  float64
dtypes: float64(10)
memory usage: 63.1+ MB


### 3.4 Endpoint 4

+ def **best_developer_year( *`año` : int* )**:
   Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
  
Ejemplo de retorno:
```json
[
  {"Puesto 1" : X},
  {"Puesto 2" : Y},
  {"Puesto 3" : Z}
]
```

* Creamos subsets de los dataframe con solo las columnas necesarias.

In [65]:
df_reviews = df_user_reviews[['item_id', 'recommend', 'posted_year', 'sentiment_analysis']]
df_games = df_endpoint1[['item_id', 'developer']]

* Luego, hacemos merge en 'item_id'

In [66]:
df_endpoint4 = pd.merge(df_reviews, df_games, on='item_id')
df_endpoint4.head()

,item_id,recommend,posted_year,sentiment_analysis,developer
0,1250,True,2011,2,Tripwire Interactive
1,1250,True,2015,2,Tripwire Interactive
2,1250,True,2013,2,Tripwire Interactive
3,1250,True,2015,1,Tripwire Interactive
4,1250,True,2014,1,Tripwire Interactive


* Por último, eliminamos 'item_id' que ya no necesitamos.

In [67]:
df_endpoint4.drop('item_id', axis=1, inplace=True)
df_endpoint4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49562 entries, 0 to 49561
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   recommend           49562 non-null  bool  
 1   posted_year         49562 non-null  object
 2   sentiment_analysis  49562 non-null  int64 
 3   developer           49562 non-null  object
dtypes: bool(1), int64(1), object(2)
memory usage: 1.6+ MB


### 3.5 Endpoint 5

+ def **developer_reviews_analysis( *`desarrolladora` : str* )**:
    Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total
    de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

Ejemplo de retorno:
```json
{
  'Valve' : [
     Negative = 182,
     Positive = 278
    ]
}
```


* Para esta funcion podemos utilizar el mismo dataset del endpoint 4.

## 4 Carga

Finalmente, en esta sección cargamos nuestros datos transformados para los endpoints que se consumirán en la API a su destino final. Optamos por almacenarlos en formato parquet con compresion snappy para reducir su tamaño de almacenamiento.

In [72]:
# Exportamos a parquet
dfs = [df_endpoint1, df_endpoint2, df_endpoint3, df_endpoint4]

# Nombres correspondientes a cada DataFrame
names = ['endpoint1', 'endpoint2', 'endpoint3', 'endpoint4']

for dfs, n in zip(dfs, names):
    path = f'data/processed/{n}.parquet'
    dfs.to_parquet(path, engine='pyarrow', compression='snappy')
    print(f"'{n}' fue guardado correctamente en '{path}'")

'endpoint1' fue guardado correctamente en 'data/processed/endpoint1.parquet'
'endpoint2' fue guardado correctamente en 'data/processed/endpoint2.parquet'
'endpoint3' fue guardado correctamente en 'data/processed/endpoint3.parquet'
'endpoint4' fue guardado correctamente en 'data/processed/endpoint4.parquet'
